# UI for the Model

In [8]:
import tensorflow as tf
final_model = tf.keras.models.load_model('surface-crack\content\surface-crack.keras')
final_model
# for layer in final_model.layers:
#     print(layer.name)

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\comp\AppData\Local\Temp\ipykernel_19216\939205353.py:2: SyntaxWarning: invalid escape sequence '\c'
  final_model = tf.keras.models.load_model('surface-crack\content\surface-crack.keras')


<Functional name=functional, built=True>

In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.cm as cm
from tensorflow.keras.preprocessing.image import img_to_array

# Load trained model

# Get the last convolutional layer
last_conv_layer_name = "conv2d"  # Change this if needed


# Function to compute Grad-CAM heatmap
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        predicted_class = tf.argmax(predictions[0])
        loss = predictions[:, predicted_class]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize heatmap
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    return heatmap, predicted_class.numpy()

# Function to apply segmentation on heatmap
PIXELS_PER_MM = 3.78  

def segment_crack(img, heatmap):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)

    _, binary_mask = cv2.threshold(heatmap, 100, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    segmented_image = img.copy()
    max_length, max_breadth = 0, 0

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w * h > max_length * max_breadth:  # largest area
            max_length, max_breadth = h, w  # height is length, width is breadth

        cv2.drawContours(segmented_image, [contour], -1, (0, 255, 0), 2)

    length_mm = round(max_length / PIXELS_PER_MM, 2)
    breadth_mm = round(max_breadth / PIXELS_PER_MM, 2)

    return segmented_image, length_mm, breadth_mm



# Function for Gradio prediction
def predict_input_image(image):
    try:
        image = np.array(image)

        # Preprocess image
        image_resized = cv2.resize(image, (224, 224))
        img_array = img_to_array(image_resized) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Get Grad-CAM heatmap
        heatmap, predicted_class = make_gradcam_heatmap(img_array, final_model, last_conv_layer_name)

        # Overlay segmentation
        segmented_image, length_mm, breadth_mm = segment_crack(image_resized, heatmap)


        # Class labels
        CLASS_LABELS = {0: "Surface clean, no cracks found.", 1: "Crack Detected"}
        confidence = final_model.predict(img_array)[0][predicted_class]
        if confidence < 0.2:
            confidence = 1 - confidence
            predicted_class = 1


        return CLASS_LABELS[predicted_class], f"Confidence: {confidence:.2%}", segmented_image, f"{length_mm} mm", f"{breadth_mm} mm"

    except Exception as e:
        return "Error", str(e), None

# Setup Gradio Interface
label = gr.Label()
confidence = gr.Textbox()
segmented_output = gr.Image(type="numpy")
length = gr.Textbox(label="Crack Length (mm)")
# breadth = gr.Textbox(label="Crack Breadth (mm)")

inter = gr.Interface(
    fn=predict_input_image,
    inputs=image,
    outputs=[label, confidence, segmented_output, length],
    title="Crack Detection with Segmentation and Measurement",
    live=True
)


inter.launch(share=True)




In [9]:
import gradio as gr
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.cm as cm
from tensorflow.keras.preprocessing.image import img_to_array

# Load trained model

# Get the last convolutional layer
last_conv_layer_name = "conv2d"  # Change this if needed


# Function to compute Grad-CAM heatmap
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        predicted_class = tf.argmax(predictions[0])
        loss = predictions[:, predicted_class]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize heatmap
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    return heatmap, predicted_class.numpy()

# Function to apply segmentation on heatmap
def segment_crack(img, heatmap):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)  # Scale heatmap to 0-255

    # No need for cvtColor, heatmap is already single-channel
    gray_heatmap = heatmap

    # Apply adaptive thresholding to segment the crack
    _, binary_mask = cv2.threshold(gray_heatmap, 100, 255, cv2.THRESH_BINARY)

    # Find contours of the crack
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the original image
    segmented_image = img.copy()
    cv2.drawContours(segmented_image, contours, -1, (0, 255, 0), 2)  # Green contours

    return segmented_image


# Function for Gradio prediction
def predict_input_image(image):
    try:
        image = np.array(image)

        # Preprocess image
        image_resized = cv2.resize(image, (224, 224))
        img_array = img_to_array(image_resized) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Get Grad-CAM heatmap
        heatmap, predicted_class = make_gradcam_heatmap(img_array, final_model, last_conv_layer_name)

        # Overlay segmentation
        segmented_image = segment_crack(image_resized, heatmap)

        # Class labels
        CLASS_LABELS = {0: "Surface clean, no cracks found.", 1: "Crack Detected"}
        confidence = final_model.predict(img_array)[0][predicted_class]
        if confidence < 0.2:
            confidence = 1 - confidence
            predicted_class = 1


        return CLASS_LABELS[predicted_class], f"Confidence: {confidence:.2%}", segmented_image

    except Exception as e:
        return "Error", str(e), None

# Setup Gradio Interface
image = gr.Image(type="numpy")
label = gr.Label()
confidence = gr.Textbox()
segmented_output = gr.Image(type="numpy")

inter = gr.Interface(
    fn=predict_input_image,
    inputs=image,
    outputs=[label, confidence, segmented_output],
    title="Crack Detection with Segmentation",
    live=True
)

inter.launch(share=True)




* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://e94ae761729c94f5ed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


d:\FinalPBLProj\crackENV\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['input_layer']]
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


d:\FinalPBLProj\crackENV\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['input_layer']]
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
